In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree


from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [14]:
data = pd.read_csv("../raw_data/X_y_data2 (2).csv") #data dims: 2184 × 80
data = data[data.y < 2] # data dims: 857 × 80
X = data.iloc[:, :-1]
y = data.iloc[:, -1:] # y = synanymous column

,y
1,1
2,1
3,1
5,1
6,1
...,...
2170,1
2177,1
2178,0
2181,0


## Dropping columns with too many NANs

In [108]:
X.head(10)

,Organization Name,IPO Status,Number of Founders,Company Type,CB Rank (Company),Operating Status,Industry Groups,Number of Employees,Estimated Revenue Range,Number of Exits,Number of Sub-Orgs,city,state,funding_status,revenue_range,no_employees,lat,lon,no_founders,industry,industry_groups,industry_groups_andreas,website,phone,email,linkedin,twitter,facebook,articles,founded_year,last_funding_year,exit_year,closed_year,no_investors,no_fund_rounds,acquired,private_ipo,company_type,operting_status,no_lead_investors,no_sub_orgs,no_exits,has_preseed,has_seed,has_series_a,has_series_b,has_series_c,has_series_d,has_series_e,has_angel,has_debt_financing,has_grant,has_corporate_round,has_series_x,has_ico,has_other_funding,preseed_fund,seed_fund,series_a_fund,series_b_fund,series_c_fund,series_d_fund,series_e_fund,private_equity_fund,seed_to_pre_ratio,a_to_seed_ratio,b_to_a_ratio,c_to_b_ratio,d_to_c_ratio,e_to_d_ratio,pe_to_e_ratio,total_funding_usd,last_funding_type,last_funding_amount,hub_tags,industry_multiples,company_valuation,months_since_founded
1,infinite roots,Private,1.0,For Profit,"1,875",Active,"Biotechnology, Food and Beverage, Health Care,...",51-100,NaN,NaN,NaN,Hamburg,Hamburg,Early Stage Venture,$1M to $10M,51-100,53.5511,9.9937,1.0,"Biotechnology, Food and Beverage, Health Care,...",Healthcare and Biotechnology,-1,1,0,1,1,0,0,21.0,2018,2024.0,NaN,NaN,14.0,8.0,0,1,1,1,4.0,0.0,0.0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,NaN,NaN,10000000.0,53000000.0,NaN,NaN,NaN,NaN,0.0,0.000000,5.300000,0.000000,0.000000,0.0,0.0,86661948.0,Series B,57481860.0,NaN,3.0,2.599858e+08,77
2,instagrid,Private,2.0,For Profit,"1,950",Active,"Energy, Manufacturing",101-250,$1M to $10M,NaN,NaN,Stuttgart,Baden-Wurttemberg,Late Stage Venture,$1M to $10M,101-250,48.7775,9.1800,2.0,"Energy, Manufacturing",Energy and Natural Resources,-1,1,1,1,1,1,1,31.0,2018,2024.0,NaN,NaN,10.0,4.0,0,1,1,1,4.0,0.0,0.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN,8500000.0,29000000.0,87430000.0,NaN,NaN,NaN,0.0,0.000000,3.411765,3.014828,0.000000,0.0,0.0,137401184.0,Series C,94927569.0,NaN,1.5,2.061018e+08,77
3,isar aerospace,Private,3.0,For Profit,"2,015",Active,"Information Technology, Science and Engineerin...",251-500,$10M to $50M,NaN,NaN,Munich,Bayern,Late Stage Venture,$10M to $50M,251-500,48.1374,11.5755,3.0,"Information Technology, Science and Engineerin...",Technology and Software,-1,1,0,1,1,1,1,52.0,2018,2024.0,NaN,NaN,16.0,5.0,0,1,1,1,5.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,15000000.0,75000000.0,NaN,NaN,NaN,NaN,0.0,0.000000,5.000000,0.000000,0.000000,0.0,0.0,350641701.0,Series C,NaN,NaN,4.0,1.402567e+09,77
5,wandelbots,Private,8.0,For Profit,"2,128",Active,"Artificial Intelligence (AI), Consumer Electro...",101-250,$1M to $10M,NaN,NaN,Dresden,Sachsen,Late Stage Venture,$1M to $10M,101-250,51.0504,13.7373,8.0,"Artificial Intelligence (AI), Consumer Electro...",Technology and Software,-1,1,1,1,1,1,1,24.0,2017,2022.0,NaN,NaN,13.0,6.0,0,1,1,1,6.0,0.0,0.0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,NaN,NaN,6000000.0,30000000.0,84000000.0,NaN,NaN,NaN,0.0,0.000000,5.000000,2.800000,0.000000,0.0,0.0,122184409.0,Series C,84000000.0,NaN,4.0,4.887376e+08,80
6,agile robots ag,Private,3.0,For Profit,"2,553",Active,"Artificial Intelligence (AI), Data and Analyti...",501-1000,$50M to $100M,NaN,NaN,Gilching,Bayern,M&A,$50M to $100M,501-1000,48.1067,11.3000,3.0,"Artificial Intelligence (AI), Data and Analyti...",Technology and Software,-1,1,1,1,1,1,1,6.0,2018,2022.0,NaN,NaN,15.0,5.0,0,1,1,1,5.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,220000000.0,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,270050589.0,Venture - Series Unknown,30000000.0,Unicorn,4.0,1.080202e+09,77
7,atai life sciences,Public,4.0,For Profit,"2,728",Active,"Biotechnology, Health Care, Science and Engine...",101-250,Less than $1M,2.0,NaN,Munich,Bayern,IPO,Less than $1M,101-250,48.1374,11.5755,4.0,"Biotechnology, Health Care, Science and Engine...",Healthcare and Biotechnology,-1,1,1,1,1,1,1,81.0,2018,2022.0,2021.0,N

In [21]:
# Let's check out no of NANs per column
pd.options.display.max_columns = 100
pd.DataFrame(X.isna().sum()).T

,Unnamed: 0,Organization Name,IPO Status,Number of Founders,Company Type,CB Rank (Company),Operating Status,Industry Groups,Number of Employees,Estimated Revenue Range,Number of Exits,Number of Sub-Orgs,city,state,funding_status,revenue_range,no_employees,lat,lon,no_founders,industry,industry_groups,industry_groups_andreas,website,phone,email,linkedin,twitter,facebook,articles,founded_year,last_funding_year,exit_year,closed_year,no_investors,no_fund_rounds,acquired,private_ipo,company_type,operting_status,no_lead_investors,no_sub_orgs,no_exits,has_preseed,has_seed,has_series_a,has_series_b,has_series_c,has_series_d,has_series_e,has_angel,has_debt_financing,has_grant,has_corporate_round,has_series_x,has_ico,has_other_funding,preseed_fund,seed_fund,series_a_fund,series_b_fund,series_c_fund,series_d_fund,series_e_fund,private_equity_fund,seed_to_pre_ratio,a_to_seed_ratio,b_to_a_ratio,c_to_b_ratio,d_to_c_ratio,e_to_d_ratio,pe_to_e_ratio,total_funding_usd,last_funding_type,last_funding_amount,hub_tags,industry_multiples,company_valuation,months_since_founded
0,0,0,0,49,0,0,0,3,0,144,850,835,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,552,801,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,675,371,419,630,766,812,836,814,0,0,0,0,0,0,0,0,1,132,806,18,18,0


In [27]:
X.drop(columns=["Unnamed: 0"], inplace=True)
X.shape #out: (857, 78)

(857, 78)

## Now let's decide what type of variable each feature is

In [38]:
X.head(3)

,Organization Name,IPO Status,Number of Founders,Company Type,CB Rank (Company),Operating Status,Industry Groups,Number of Employees,Estimated Revenue Range,Number of Exits,Number of Sub-Orgs,city,state,funding_status,revenue_range,no_employees,lat,lon,no_founders,industry,industry_groups,industry_groups_andreas,website,phone,email,linkedin,twitter,facebook,articles,founded_year,last_funding_year,exit_year,closed_year,no_investors,no_fund_rounds,acquired,private_ipo,company_type,operting_status,no_lead_investors,no_sub_orgs,no_exits,has_preseed,has_seed,has_series_a,has_series_b,has_series_c,has_series_d,has_series_e,has_angel,has_debt_financing,has_grant,has_corporate_round,has_series_x,has_ico,has_other_funding,preseed_fund,seed_fund,series_a_fund,series_b_fund,series_c_fund,series_d_fund,series_e_fund,private_equity_fund,seed_to_pre_ratio,a_to_seed_ratio,b_to_a_ratio,c_to_b_ratio,d_to_c_ratio,e_to_d_ratio,pe_to_e_ratio,total_funding_usd,last_funding_type,last_funding_amount,hub_tags,industry_multiples,company_valuation,months_since_founded
1,infinite roots,Private,1.0,For Profit,"1,875",Active,"Biotechnology, Food and Beverage, Health Care,...",51-100,NaN,NaN,NaN,Hamburg,Hamburg,Early Stage Venture,$1M to $10M,51-100,53.5511,9.9937,1.0,"Biotechnology, Food and Beverage, Health Care,...",Healthcare and Biotechnology,-1,1,0,1,1,0,0,21.0,2018,2024.0,NaN,NaN,14.0,8.0,0,1,1,1,4.0,0.0,0.0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,NaN,NaN,10000000.0,53000000.0,NaN,NaN,NaN,NaN,0.0,0.0,5.300000,0.000000,0.0,0.0,0.0,86661948.0,Series B,57481860.0,NaN,3.0,2.599858e+08,77
2,instagrid,Private,2.0,For Profit,"1,950",Active,"Energy, Manufacturing",101-250,$1M to $10M,NaN,NaN,Stuttgart,Baden-Wurttemberg,Late Stage Venture,$1M to $10M,101-250,48.7775,9.1800,2.0,"Energy, Manufacturing",Energy and Natural Resources,-1,1,1,1,1,1,1,31.0,2018,2024.0,NaN,NaN,10.0,4.0,0,1,1,1,4.0,0.0,0.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN,8500000.0,29000000.0,87430000.0,NaN,NaN,NaN,0.0,0.0,3.411765,3.014828,0.0,0.0,0.0,137401184.0,Series C,94927569.0,NaN,1.5,2.061018e+08,77
3,isar aerospace,Private,3.0,For Profit,"2,015",Active,"Information Technology, Science and Engineerin...",251-500,$10M to $50M,NaN,NaN,Munich,Bayern,Late Stage Venture,$10M to $50M,251-500,48.1374,11.5755,3.0,"Information Technology, Science and Engineerin...",Technology and Software,-1,1,0,1,1,1,1,52.0,2018,2024.0,NaN,NaN,16.0,5.0,0,1,1,1,5.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,15000000.0,75000000.0,NaN,NaN,NaN,NaN,0.0,0.0,5.000000,0.000000,0.0,0.0,0.0,350641701.0,Series C,NaN,NaN,4.0,1.402567e+09,77


In [56]:
### Berkan categorisations
one_hot_category = [ "funding_status", "industry_groups"]

ordinal_category = ["no_employees", "revenue_range"]

numerical_features = ["no_founders",
                      'no_investors',
                      'no_fund_rounds',
                      'no_sub_orgs',
                      'has_preseed',
                      'has_seed',
                      'has_series_a',
                      'has_series_b',
                      'has_series_c',
                      'has_debt_financing',
                      'has_grant',
                      'has_series_x',
                      'months_since_founded',
                      'lat',
                      'lon']

In [142]:
np.random.randint(2)

1

In [121]:
data['funding_status'].value_counts().index

Index(['M&A', 'Early Stage Venture', 'Seed', 'Late Stage Venture',
       'Private Equity', 'IPO'],
      dtype='object')

## We need to replace "-1" with "Other" in "industry_groups" columns

# Which features can we remove because too imbalanced

In [118]:
# data['private_ipo'].value_counts()
# data['linkedin'].value_counts() # also: facebook, twitter, linkedin, email, phone, website
# data['has_series_d'].value_counts()
# data['has_series_e'].value_counts()
# data['has_angel'].value_counts()
# data['has_corporate_round'].value_counts()
# data['operting_status'].value_counts()


1    768
0     89
Name: operting_status, dtype: int64

# Continue model  

In [58]:
no_employees_ordinal = [
    '11-50', '51-100', '101-250', '251-500', '501-1000', '1001-5000', '5001-10000', '10001+'
]

In [59]:
revenue_range_ordinal = [
    'Less than $1M', '$1M to $10M', '$10M to $50M', '$50M to $100M', '$100M to $500M', '$500M to $1B', '$1B to $10B', '$10B+'
]

In [60]:
feat_ordinal_dict = {
    "no_employees": no_employees_ordinal,
    "revenue_range": revenue_range_ordinal
}

In [62]:
encoder_ordinal = OrdinalEncoder(
    categories = [feat_ordinal_dict[i] for i in ordinal_category],
    dtype = np.int64
)


preproc_ordinal = make_pipeline(
    SimpleImputer(strategy = "most_frequent"),
    encoder_ordinal,
    MinMaxScaler()
)

preproc_ordinal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['11-50', '51-100', '101-250',
                                             '251-500', '501-1000', '1001-5000',
                                             '5001-10000', '10001+'],
                                            ['Less than $1M', '$1M to $10M',
                                             '$10M to $50M', '$50M to $100M',
                                             '$100M to $500M', '$500M to $1B',
                                             '$1B to $10B', '$10B+']],
                                dtype=<class 'numpy.int64'>)),
                ('minmaxscaler', MinMaxScaler())])

In [63]:
preproc_min_numerical = make_pipeline(
    KNNImputer(),
    MinMaxScaler())

preproc_min_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('minmaxscaler', MinMaxScaler())])

In [64]:
preproc_nominal = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preproc_nominal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [65]:
preproc_robust_numerical = make_pipeline(
    KNNImputer(),
    RobustScaler())

preproc_robust_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('robustscaler', RobustScaler())])

In [66]:
preproc = make_column_transformer(
        (preproc_ordinal, ordinal_category),
        (preproc_min_numerical, numerical_features),
        (preproc_nominal, one_hot_category),
        # (preproc_robust_numerical, robust_category),
        remainder="drop"
)

In [67]:
preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['11-50',
                                                                              '51-100',
                                                                              '101-250',
                                                                              '251-500',
                                                                              '501-1000',
                                                                              '1001-5000',
                                                                              '5001-10000',
                                                                              '10001+'],
                                                                             ['Less '
                                                                              'than '
                                                                              '$1M',
                                                                              '$1M '
                                                                              'to '
                                                                              '$10M',
                                                                              '$10M '
                                                                              'to '
                                                                              '$50M',
                                                                              '$50M '
                                                                              'to '
                                                                              '$100M',
                                                                              '$100M '
                                                                              'to '
                                                                              '$500M',
                                                                              '$500M '
                                                                              'to '
                                                                              '...
                                  'has_series_c', 'has_series_d',
                                  'has_series_e', 'has_angel',
                                  'has_debt_financing', 'has_grant',
                                  'has_corporate_round', 'has_series_x',
                                  'months_since_founded']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['state', 'funding_status', 'industry_groups',
                                  'private_ipo'])])

In [68]:
X_preprocessed = pd.DataFrame(preproc.fit_transform(X, y))

In [69]:
X_preprocessed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70
0,0.142857,0.142857,0.642857,1.0,1.0,0.0,0.000,1.0,1.0,0.0,0.0,0.304348,0.333333,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.360947,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.285714,0.142857,0.642857,1.0,1.0,1.0,0.125,1.0,1.0,1.0,1.0,0.217391,0.142857,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360947,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.428571,0.285714,0.642857,1.0,1.0,0.0,0.250,1.0,1.0,1.0,1.0,0.347826,0.190476,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360947,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.285714,0.142857,0.571429,1.0,1.0,1.0,0.875,1.0,1.0,1.0,1.0,0.282609,0.238095,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.378698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.571429,0.428571,0.642857,1.0,1.0,1.0,0.250,1.0,1.0,1.0,1.0,0.326087,0.190476,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.360947,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,0.000000,0.142857,0.357143,1.0,1.0,1.0,0.250,1.0,1.0,1.0,1.0,0.173913,0.238095,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.644970,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
853,0.000000,0.000000,0.357143,1.0,1.0,1.0,0.125,1.0,1.0,1.0,0.0,0.043478,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.621302,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
854,0.000000,0.142857,0.357143,1.0,1.0,0.0,0.125,1.0,1.0,1.0,1.0,0.000000,0.142857,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.573964,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
855,0.000000,0.142857,0.357143,1.0,1.0,0.0,0.125,0.0,0.0,0.0,0.0,0.021739,0.095238,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.644970,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [70]:
#X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y.iloc[:, -1:], test_size=.20, random_state=1)

# STOP HERE!!
# Everything below is Berkan's

In [71]:
! pip install tensorflow

In [72]:
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping

In [73]:
X_preprocessed.shape

(857, 71)

In [74]:
reg = regularizers.l1_l2(l2=0.005)

model = Sequential()
model.add(layers.Input(shape=(X_preprocessed.shape[1],)))
model.add(layers.Dense(50, activation="relu", kernel_regularizer=reg))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(20, activation="tanh"))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

In [75]:
learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [76]:
es = EarlyStopping(
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=700,
    batch_size=32,
    callbacks=[es],
    verbose=0
)

Epoch 52: early stopping
Restoring model weights from the end of the best epoch: 32.


In [77]:
history.history

{'accuracy': [0.5182482004165649,
  0.6094890236854553,
  0.6478102207183838,
  0.7025547623634338,
  0.7572992444038391,
  0.7773722410202026,
  0.790145993232727,
  0.8229926824569702,
  0.8467153310775757,
  0.8759124279022217,
  0.8886861205101013,
  0.8722627758979797,
  0.9014598727226257,
  0.9014598727226257,
  0.9288321137428284,
  0.9160584211349487,
  0.9324817657470703,
  0.9105839133262634,
  0.9288321137428284,
  0.9361313581466675,
  0.9288321137428284,
  0.94525545835495,
  0.9379562139511108,
  0.9434306621551514,
  0.9361313581466675,
  0.9324817657470703,
  0.9470803141593933,
  0.9507299065589905,
  0.9397810101509094,
  0.9434306621551514,
  0.9397810101509094,
  0.9416058659553528,
  0.9543795585632324,
  0.9470803141593933,
  0.9543795585632324,
  0.9525547623634338,
  0.9525547623634338,
  0.9525547623634338,
  0.956204354763031,
  0.9635036587715149,
  0.9635036587715149,
  0.959854006767273,
  0.959854006767273,
  0.9616788029670715,
  0.9507299065589905,
  0.

In [78]:
results = model.evaluate(X_test, y_test)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9008 - loss: 0.3788 


In [ ]:
model.save("../models/first_dummy.keras")